In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from skimage import io
from pywt import dwt2

def extract_features(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # DCT
    dct_features = cv2.dct(np.float32(gray) / 255.0)
    dct_features = dct_features.flatten()[:100]  # Take the top 100 coefficients
    
    # Wavelet Transform
    coeffs2 = dwt2(gray, 'haar')
    cA, (cH, cV, cD) = coeffs2
    wavelet_features = np.concatenate([cA.flatten(), cH.flatten(), cV.flatten(), cD.flatten()])[:100]
    
    # FFT
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    epsilon = 1e-10  # To avoid log(0)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + epsilon)
    fft_features = magnitude_spectrum.flatten()[:100]  # Take the top 100 coefficients
    
    # Combine features
    features = np.concatenate([dct_features, wavelet_features, fft_features])
    
    # Check for NaN or infinite values
    if np.any(np.isnan(features)) or np.any(np.isinf(features)):
        return None  # Return None for invalid features
    
    return features

# Load dataset
def load_data(base_path):
    features, labels = [], []
    for label in ['real', 'fake']:
        folder_path = os.path.join(base_path, label)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            feature_vector = extract_features(image)
            if feature_vector is not None:  # Only append valid feature vectors
                features.append(feature_vector)
                labels.append(0 if label == 'real' else 1)  # 0 for real, 1 for fake
    return np.array(features), np.array(labels)

# Prepare data
base_path = r"C:\Desktop\ML_Implementation\data(Final_ML)\train"
X, y = load_data(base_path)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import uniform, randint

# Define the Perceptron class compatible with scikit-learn
class Perceptron(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate=0.01, n_iters=1000, random_state=None):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.random_state = random_state
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        # Initialize parameters
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Training with optional shuffling for randomness
        rng = np.random.RandomState(self.random_state)
        for _ in range(self.n_iters):
            indices = rng.permutation(n_samples)
            for idx in indices:
                x_i = X[idx]
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self._activation_function(linear_output)
                # Update rule
                update = self.learning_rate * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update
        return self

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self._activation_function(linear_output)
        return y_predicted

    def _activation_function(self, x):
        return np.where(x >= 0, 1, 0)  # Step activation function

# Example usage with RandomizedSearchCV
if __name__ == "__main__":
    from sklearn.datasets import make_classification

    # Generate a synthetic binary classification dataset
    X, y = make_classification(n_samples=1000, n_features=20, 
                               n_informative=15, n_redundant=5, 
                               random_state=42)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Initialize the Perceptron
    perceptron = Perceptron(random_state=42)

    # Define the hyperparameter distributions
    param_dist = {
        'learning_rate': uniform(0.001, 0.1),  # Continuous uniform distribution
        'n_iters': randint(100, 2000),        # Integer uniform distribution
    }

    # Set up RandomizedSearchCV
    randomized_search = RandomizedSearchCV(
        estimator=perceptron,
        param_distributions=param_dist,
        n_iter=50,                # Number of parameter settings sampled
        cv=5,                     # 5-fold cross-validation
        scoring='accuracy',       # Evaluation metric
        random_state=42,
        n_jobs=-1,                # Use all available cores
        verbose=1
    )

    # Perform the hyperparameter search
    randomized_search.fit(X_train, y_train)

    # Retrieve the best estimator
    best_perceptron = randomized_search.best_estimator_

    print("Best Hyperparameters:")
    print(randomized_search.best_params_)

    # Make predictions with the best estimator
    y_pred = best_perceptron.predict(X_test)

    # Evaluate performance
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits


C:\Users\ru368\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_search.py:979: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best Hyperparameters:
{'learning_rate': 0.03845401188473625, 'n_iters': 960}

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.78      0.80       106
           1       0.77      0.81      0.79        94

    accuracy                           0.80       200
   macro avg       0.79      0.80      0.79       200
weighted avg       0.80      0.80      0.80       200

Confusion Matrix:
[[83 23]
 [18 76]]


In [7]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform, randint

# Define the Perceptron class compatible with scikit-learn
# [This section is removed since we are using MLPClassifier instead]

if __name__ == "__main__":
    from sklearn.datasets import make_classification

    # Generate a synthetic binary classification dataset
    X, y = make_classification(n_samples=1000, n_features=20, 
                               n_informative=15, n_redundant=5, 
                               random_state=42)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Define the hyperparameter distributions
    param_dist = {
        'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100), (150, 100, 50)],
        'activation': ['relu', 'tanh', 'logistic'],
        'solver': ['adam', 'sgd', 'lbfgs'],
        'alpha': uniform(0.0001, 0.01),             # L2 penalty (regularization term)
        'learning_rate_init': uniform(0.0001, 0.01),
        'max_iter': randint(200, 1000),
        'momentum': uniform(0.0, 0.9),              # Only relevant for 'sgd'
    }

    # Initialize the MLPClassifier
    mlp = MLPClassifier(random_state=42)

    # Set up RandomizedSearchCV
    randomized_search = RandomizedSearchCV(
        estimator=mlp,
        param_distributions=param_dist,
        n_iter=50,                # Number of parameter settings sampled
        cv=5,                     # 5-fold cross-validation
        scoring='accuracy',       # Evaluation metric
        random_state=42,
        n_jobs=-1,                # Use all available cores
        verbose=1
    )

    # Perform the hyperparameter search
    randomized_search.fit(X_train, y_train)

    # Retrieve the best estimator
    best_mlp = randomized_search.best_estimator_

    print("Best Hyperparameters:")
    print(randomized_search.best_params_)

    # Make predictions with the best estimator
    y_pred = best_mlp.predict(X_test)

    # Evaluate performance
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# Modify the MLPClassifier initialization
mlp = MLPClassifier(
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,    # 10% of training data used as validation
    n_iter_no_change=10         # Number of iterations with no improvement to wait before stopping
)



Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters:
{'activation': 'relu', 'alpha': 0.005126370931051921, 'hidden_layer_sizes': (100, 100), 'learning_rate_init': 0.0067892405966309965, 'max_iter': 680, 'momentum': 0.17571868901824006, 'solver': 'adam'}

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       100
           1       0.94      0.97      0.96       100

    accuracy                           0.95       200
   macro avg       0.96      0.95      0.95       200
weighted avg       0.96      0.95      0.95       200

Confusion Matrix:
[[94  6]
 [ 3 97]]
